<a href="https://colab.research.google.com/github/jhkang0516/dialect_classification/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 필요한 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# 모든 데이터

In [ ]:
dataDir = '/content/drive/MyDrive/voice_data/'

X_mfcc_ALL = np.load(dataDir + 'X_mfcc_ALL.npy')
Y_label_ALL = np.load(dataDir + 'Y_label_ALL.npy')

In [ ]:
print(X_mfcc_ALL.shape)
print(Y_label_ALL.shape)

(60000, 39, 259)
(60000,)


In [ ]:
Y_label_df = pd.DataFrame(Y_label_ALL)
Y_label_df

,0
0,경상
1,경상
2,경상
3,경상
4,경상
...,...
59995,표준
59996,표준
59997,표준
59998,표준


In [ ]:
# 데이터 변환(3차원 -> 2차원)
X_mfcc_ALL = np.mean(X_mfcc_ALL, axis=1)

In [ ]:
print(X_mfcc_ALL.shape)

(60000, 259)


In [ ]:
# Y_label_ALL 라벨인코딩-원핫인코딩
DF_Y = pd.DataFrame(Y_label_ALL)
DF_Y = pd.get_dummies(DF_Y)

In [ ]:
# 인코딩 결과 확인
DF_Y

,0_강원,0_경상,0_전라,0_제주,0_충청,0_표준
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
59995,0,0,0,0,0,1
59996,0,0,0,0,0,1
59997,0,0,0,0,0,1
59998,0,0,0,0,0,1


In [ ]:
# 학습 데이터 / 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_mfcc_ALL, Y_label_ALL, test_size=0.3, random_state=42)

In [ ]:
# 탐색할 kernel과 C 값들
# kernels = ['linear', 'rbf', 'poly']
kernels = ['rbf', 'poly']
C_values = [0.1, 1, 10]

# 최적의 조합을 저장할 변수 초기화
best_score = 0
best_kernel = ''
best_C = 0

# 그리디 서치
for kernel in kernels:
    for C in C_values:
        print(kernel);
        svm_model = svm.SVC(kernel=kernel, C=C)
        svm_model.fit(X_train, y_train)
        accuracy = svm_model.score(X_test, y_test)

        # 더 높은 정확도를 찾으면 최적의 조합 갱신
        if accuracy > best_score:
            best_score = accuracy
            best_kernel = kernel
            best_C = C

# 최적의 조합 출력
print("Best kernel:", best_kernel)
print("Best C:", best_C)
print("Best accuracy:", best_score)

rbf
rbf
rbf
poly
poly
poly
Best kernel: rbf
Best C: 1
Best accuracy: 0.4872222222222222


In [ ]:
# 최적의 커널 함수, c 값을 통한 예측값
svm_model = svm.SVC(kernel=best_kernel, C=best_C)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

In [ ]:
# Precision, Recall, F1-Score
report = classification_report(y_test, svm_predictions)
print(report)

              precision    recall  f1-score   support

          강원       0.43      0.66      0.52      2942
          경상       0.42      0.23      0.30      3065
          전라       0.44      0.38      0.41      2949
          제주       0.45      0.74      0.56      2930
          충청       0.64      0.47      0.55      3057
          표준       0.63      0.44      0.52      3057

    accuracy                           0.49     18000
   macro avg       0.50      0.49      0.48     18000
weighted avg       0.50      0.49      0.48     18000

